In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# %cd /content/drive/MyDrive/Sun/ML Shock/Final project/
import sys
sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")

In [11]:
from torch.utils.data import DataLoader
import pandas as pd
import os
import numpy as np

# import CustomDataset from CustomDataset.py from utils folder
from CustomDataset import CustomDataset

In [12]:
# Load (train or test) data from csv file
path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/sample_data/"
# path = "./data/"

data_type = "spec" # "eeg_raw" #"eeg_spec"  #
train = True
text = "train" if train else "test"


In [ ]:
df = pd.read_csv(path + f"{text}.csv")

In [13]:

data_dir = (
    f"{text}_eegs/"
    if "eeg" in data_type
    else f"{text}_spectrograms/"
)
data_dir = path + data_dir

votes_cols = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]
label_cols = (
    ["eeg_id", "label_id", "eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_id", "label_id", "spectrogram_label_offset_seconds"]
)
offset = (
    ["eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_label_offset_seconds"]
)


In [14]:
files = os.listdir(data_dir)
df = df[
    df["eeg_id" if "eeg" in data_type else "spectrogram_id"].isin(
        [int(f.split(".")[0]) for f in files]
    )
]

In [15]:
# if info_cols not in df add it and set to zero
for col in offset:
    if col not in df.columns:
        df[col] = 0
# if df does not contain "label_id" add a unique label_id
if "label_id" not in df.columns:
    df["label_id"] = range(len(df))

info = {}
df_gr = df.groupby(label_cols)
for name, group in df_gr:
    # first row of group
    info[name] = {"votes": group[votes_cols].values[0] if train else None}


In [16]:
dataset = CustomDataset(data_dir, data_type, info)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [17]:
count = 0
for data, label, class_probs in dataloader:
    print(data.shape, label.shape, class_probs.shape)
    count += 1
    if count > 0:
        break

torch.Size([16, 299, 100, 4]) torch.Size([16]) torch.Size([16, 6])


In [18]:
# create a data scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

count = 0
# use the partial function to create a scaler
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
for data, _, _ in dataloader:
  try:
    scale = standard_scaler.scale_
    min = min_max_scaler.data_min_
    max = min_max_scaler.data_max_
  except:
    scale = None
  standard_scaler.partial_fit(data.reshape(data.shape[0],-1))
  min_max_scaler.partial_fit(data.reshape(data.shape[0],-1))
  scale_new = standard_scaler.scale_
  min_new = min_max_scaler.data_min_
  max_new = min_max_scaler.data_max_
  if scale is not None:
    change = np.abs(scale_new - scale)
    change_minmax = np.abs([min_new - min, max_new - max])
    print(f"Std: {change.max():.4e} MinMax: {change_minmax.max():.4e}")
  count +=1

  if count%10 == 0:
    # with open(path +f'{data_type}_standard_scaler.pkl', 'wb') as f:
    #   pickle.dump(standard_scaler, f)
    # with open(path + f'{data_type}_min_max_scaler.pkl', 'wb') as f:
    #     pickle.dump(min_max_scaler, f)
    print("saved", count)


Std: 1.7311e+00 MinMax: 1.1312e+01
Std: 1.3828e+00 MinMax: 1.1130e+01
Std: 1.0994e+00 MinMax: 1.0624e+01
Std: 1.0933e+00 MinMax: 1.0371e+01
